# Daily graphs of COVID-19 pandemic

- Automatically updated every day at 22:00 (Argentina Standard Time)
- Data downloaded from [John Hopkins University repository](https://github.com/CSSEGISandData/COVID-19)
- Code released under GNU GPLv3 License [github.com/epassaro/covid-19](https://github.com/epassaro/covid-19)

In [ ]:
import itertools
import numpy as np
import pandas as pd
from bokeh.plotting import figure, show, ColumnDataSource
from bokeh.models.tools import HoverTool
from bokeh.palettes import Category10
from bokeh.io import output_notebook

output_notebook()

In [ ]:
!wget -q https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv

In [ ]:
countries = {'South America': ['Argentina', 'Bolivia', 'Brazil', 
                               'Chile', 'Colombia', 'Ecuador', 
                               'Paraguay', 'Peru', 'Uruguay', 
                               'Venezuela'],
                         
             'North America': ['Canada', 'Mexico', 'US'],
             
             'Europe': ['Italy', 'Spain', 'France', 
                        'Germany', 'United Kingdom'] + ['Belgium', 
                        'Netherlands', 'Russia', 'Switzerland'],
             
             'Asia & Oceania': ['Japan', 'Singapore', 'Thailand',
                                'Indonesia', 'Philippines', 'India',
                                'Australia', 'India'],

             'World': ['China', 'Korea, South', 'Italy',
                       'Iran', 'Spain', 'Germany', 
                       'United Kingdom', 'US', 'Australia', 
                       'Brazil']}

In [ ]:
palette = itertools.cycle(Category10[10])

In [ ]:
def shift(serie):
    day_zero = serie.ne(0).idxmax()
    return serie[day_zero:]

In [ ]:
fname = './time_series_covid19_confirmed_global.csv'

In [ ]:
def plot_cases(fname, countries, key):
    
    ### Create table
    
    df = pd.read_csv(fname)
    df = df.drop(columns=['Province/State', 'Lat', 'Long'])
    df = df.groupby(by=['Country/Region']).sum()
    df = df.transpose()
    df.index.name = 'date'
    df.columns.name = None
    #df.index = pd.to_datetime(df.index)
    df = df.reset_index()

    df_list = []
    for country in countries[key]:
        _ = shift(df[country])
        _ = _.reset_index(drop=True)
        df_list.append(_)
    
    df = pd.concat(df_list, axis=1)


    ### Plot begins
    
    source = ColumnDataSource(df)
    p = figure(plot_width=680, plot_height=420)

    hovers = []
    for country in countries[key]:
    
        color = next(palette)
        p.line(x='index', y=country, source=source,
               line_width=1.5, 
               legend_label=country, 
               color=color)

        p.circle(x='index', y=country, source=source,
                 legend_label=country, 
                 color=color,
                 name=country) # Tooltip are shown only on points! 
    
        TOOLTIPS = [('Country', country),
                    ('Day', '@index'),
                    ('Total cases', '@{}'.format(country))]
    
        hover = HoverTool(tooltips=TOOLTIPS, names=[country])
        hovers.append(hover)

    p.add_tools(*hovers)
    p.legend.click_policy = 'hide'
    p.legend.location = 'top_left'
    p.legend.label_text_font_size = '8pt'
    p.yaxis.formatter.use_scientific = False
    p.xaxis.axis_label = "Days since 1st case"
    p.yaxis.axis_label = "Total cases"
    p.title.text = "Cumulative confirmed COVID-19 cases ({})".format(key)

    return show(p)

In [ ]:
_ = [ plot_cases(fname, countries, k) for k in countries.keys() ]